<a href="https://colab.research.google.com/github/tomonari-masada/course2025-stats1/blob/main/diagonal_gaussian_mixture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EM algorithm for Diagonal Gaussian mixture models

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import torch
from datasets import load_from_disk
from transformers import set_seed
from sentence_transformers import SentenceTransformer

torch.set_grad_enabled(False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

set_seed(0)

In [ ]:
!wget https://github.com/tomonari-masada/course2025-nlp/raw/refs/heads/main/livedoor_ds.tar.gz
!tar zxf livedoor_ds.tar.gz

In [ ]:
ds = load_from_disk("livedoor_ds")
category_names = [
    'movie-enter',
    'it-life-hack',
    'kaden-channel',
    'topic-news',
    'livedoor-homme',
    'peachy',
    'sports-watch',
    'dokujo-tsushin',
    'smax',
]

num_labels = len(set(ds["train"]["category"]))
print(f"Number of labels: {num_labels}")

In [ ]:
ds["train"][0]

In [ ]:
y_true = np.array(ds["train"]["category"])

In [ ]:
model_name = "cl-nagoya/ruri-v3-310m"
model = SentenceTransformer(model_name, device=device)
model.eval()

In [ ]:
embeddings = model.encode(ds["train"]["title"], show_progress_bar=True, convert_to_numpy=True)

In [ ]:
type(embeddings)

In [ ]:
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(embeddings)
cluster_assignments = kmeans.labels_

In [ ]:
def print_cluster_info(cluster_assignments, y_true, num_clusters, category_names):
    for k in range(num_clusters):
        print(f"Cluster {k}:")
        indices = np.where(cluster_assignments == k)[0]
        y_true_cluster = y_true[indices]
        unique, counts = np.unique(y_true_cluster, return_counts=True)
        label_counts = dict(zip(unique, counts))
        sorted_label_counts = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)
        for label, count in sorted_label_counts:
            print(f"  Label {label} ({category_names[label]}): {count}")
        print()

In [ ]:
print_cluster_info(cluster_assignments, y_true, n_clusters, category_names)

In [ ]:
def M_step(embeddings, q):
    n_clusters = q.shape[1]
    n_features = embeddings.shape[1]
    cluster_probs = np.zeros(n_clusters)
    means = np.zeros((n_clusters, n_features))
    variances = np.zeros((n_clusters, n_features))

    for k in range(n_clusters):
        cluster_probs[k] = q[:, k].sum() / len(embeddings)
        means[k] = (q[:, k][:, np.newaxis] * embeddings).sum(axis=0) / q[:, k].sum()
        diff = embeddings - means[k]
        variances[k] = (q[:, k][:, np.newaxis] * (diff ** 2)).sum(axis=0) / q[:, k].sum()

    return cluster_probs, means, variances

In [ ]:
def E_step(embeddings, cluster_probs, means, variances):
    n_clusters = cluster_probs.shape[0]
    n_samples = embeddings.shape[0]
    q = np.zeros((n_samples, n_clusters))

    for k in range(n_clusters):
        diff = embeddings - means[k]
        exponent = -0.5 * np.sum((diff ** 2) / variances[k], axis=1)
        log_coeff = -0.5 * (embeddings.shape[1] * np.log(2 * np.pi) + np.sum(np.log(variances[k])))
        q[:, k] = np.log(cluster_probs[k] + 1e-10) + log_coeff + exponent

    q = np.exp(q - q.max(axis=1, keepdims=True))
    q /= q.sum(axis=1, keepdims=True)
    return q

In [ ]:
def lower_bound(embeddings, cluster_probs, means, variances, q):
    n_samples, n_clusters = q.shape
    n_features = embeddings.shape[1]
    lb = 0.0

    for i in range(n_samples):
        for k in range(n_clusters):
            diff = embeddings[i] - means[k]
            exponent = -0.5 * np.sum((diff ** 2) / variances[k])
            log_coeff = -0.5 * (n_features * np.log(2 * np.pi) + np.sum(np.log(variances[k])))
            log_p_x_given_z = log_coeff + exponent
            lb += q[i, k] * (np.log(cluster_probs[k] + 1e-10) + log_p_x_given_z - np.log(q[i, k] + 1e-10))

    return lb / n_samples

In [ ]:
rng = np.random.default_rng(0)
n_samples, n_features = embeddings.shape

q = rng.uniform(size=(n_samples, n_clusters))
q /= q.sum(axis=1, keepdims=True)

In [ ]:
for epoch in range(30):
    cluster_probs, means, variances = M_step(embeddings, q)
    q = E_step(embeddings, cluster_probs, means, variances)
    lb = lower_bound(embeddings, cluster_probs, means, variances, q)
    print(f"Epoch {epoch + 1}, Lower bound: {lb:.4f}")

In [ ]:
(q.max(axis=1) == 1.0).sum()

In [ ]:
dominant_clusters = np.argmax(q, axis=1)
print_cluster_info(dominant_clusters, y_true, n_clusters, category_names)